In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 6, 5) # Input channels 3 (for RGB CIFAR10), output channels 6, kernel 5x5
        self.bn1 = nn.BatchNorm2d(6)    # Batch Normalization after conv1. Number of features matches out_channels of conv1

        self.pool = nn.MaxPool2d(2, 2)  # Max Pooling layer

        self.conv2 = nn.Conv2d(6, 16, 5) # Input channels 6 (from conv1 output), output channels 16, kernel 5x5
        self.bn2 = nn.BatchNorm2d(16)   # Batch Normalization after conv2. Number of features matches out_channels of conv2

        # Fully Connected Layers
        # The input size for fc1 is calculated: (image_size - kernel + 2*padding)/stride + 1
        # After conv1 (32x32 -> 28x28), after pool (28x28 -> 14x14)
        # After conv2 (14x14 -> 10x10), after pool (10x10 -> 5x5)
        # So, 16 channels * 5 * 5 pixels = 400 features
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 output classes for CIFAR10

        # Dropout layers for regularization
        self.dropout1 = nn.Dropout(p=0.25) # Common to put after pooling/before flattening, or in linear stack
        self.dropout2 = nn.Dropout(p=0.5)  # Another dropout in the linear stack

    def forward(self, x):
        # First Conv -> BatchNorm -> ReLU -> Pool block
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)

        # Second Conv -> BatchNorm -> ReLU -> Pool block
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)

        # Flatten all dimensions except batch
        x = torch.flatten(x, 1)

        # Dropout after flattening (optional, but can be helpful)
        x = self.dropout1(x)

        # First Linear -> ReLU
        x = self.fc1(x)
        x = F.relu(x)

        # Second Linear -> Dropout -> ReLU
        x = self.fc2(x)
        x = self.dropout2(x) # Dropout in the linear stack
        x = F.relu(x)

        # Final Linear layer (output layer, no activation here for CrossEntropyLoss)
        x = self.fc3(x)
        return x

# Re-instantiate your model after modifying the class
net = Net()

# IMPORTANT: If you are using a GPU in Colab, move the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

print(net) # Print the model architecture to see the new layers

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)  # Move data to GPU

# print images (move back to CPU for display)
imshow(torchvision.utils.make_grid(images.cpu()))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))
net.to(device)  # Move the loaded model to device

In [ ]:
outputs = net(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')